In [226]:
# %load ../import.py
import quandl
quandl.ApiConfig.api_key = 'MBMzvkxtv63KjFEV-tL6'
from quandl.errors.quandl_error import NotFoundError

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from sklearn import linear_model

import ipywidgets as widgets 
from ipywidgets import  interact, interactive, fixed, interact_manual
from IPython.display import display

#####################################################################################
# From tensorflow includes
import math
from matplotlib import cm
from matplotlib import gridspec
# from matplotlib import pyplot as plt

from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset


# For elongating day 
from datetime import timedelta

## Tensorflow GPU Test

In [227]:
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))
    
## Configrirng TF
tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows =10
pd.options.display.float_format = '{:.1f}'.format


[[22. 28.]
 [49. 64.]]


## Setting up columns name

In [228]:
s_list=pd.read_csv("../sector_list.csv")
main_col_names= [
 'Date',
 'Close',
 'TotalTradeQuantity',
 'Turnover_Lacs',
 'Return',   
 'DEP_Q',
 'DIVSH_Q',
 'EBIDT_Q',
 'EPS_Q',
 'EQCAP_Q',
 'ETR_Q',
 'FV_Q',
 'INT_Q',
 'MCAP_Q',
 'NP_Q',
 'OEXPNS_Q',
 'OI_Q',
 'OP_Q',
 'OPMSH_Q',
 'OPSH1Q_Q',
 'PBDT_Q',
 'PBT_Q',
 'REVSH_Q',
 'SHARE_Q',
 'SR_Q',
 'TAX_Q',
 'TI_Q',
 'monthday',
 'month',
 'weekday',
 'Counter',
 'Ticker' ]

## Selecting the sectors 

In [229]:
def select_sector(sector_name):
    
    sector_file_name= "../Panel/Data/"+sector_name+".csv"
    df= pd.read_csv(sector_file_name,header=None, names=main_col_names, dtype=str)
    
    return df


sector_value=widgets.Dropdown(
                            options=s_list['sector'], 
                            description="Sector:"
                            )
w=widgets.interactive(select_sector, sector_name=sector_value)
display(w)

aW50ZXJhY3RpdmUoY2hpbGRyZW49KERyb3Bkb3duKGRlc2NyaXB0aW9uPXUnU2VjdG9yOicsIG9wdGlvbnM9KCdBR1JPX0lOUFVUUycsICdBVVRPTU9CSUxFUycsICdBVVRPX0FOQ0lMTEFSSUXigKY=


In [230]:
df=w.result
df = df.apply(pd.to_numeric, errors='ignore')
df=df[df["Ticker"]=="NSE/ARIES"]

### Paneling the dataframe

In [231]:
items = df["Ticker"].unique()
minor_axis=df.columns[1:]


major_axis=df["Date"].unique()
major_axis.sort()

### Panels:
#### p = Panel
* p3 = all exluding last 7
* p4 = only last 7
* p5 = all

In [232]:
p3=pd.Panel(major_axis=major_axis[:-7], minor_axis=minor_axis)
p4=pd.Panel(major_axis=major_axis[-7:], minor_axis=minor_axis)
p5=pd.Panel(major_axis=major_axis, minor_axis=minor_axis)


for i in items:
    p3_data=df[df["Ticker"]==i]
    p3_data.set_index("Date", inplace=True)
    p3[i]=p3_data[:"2018-04-11"]
    p4[i]=p3_data["2018-04-12":]
    p5[i]=p3_data

    

## Merging All tickers

In [233]:
sector_name=w.kwargs["sector_name"]

p3_file_name= "Slices/p3_"+sector_name+".csv"
p4_file_name= "Slices/p4_"+sector_name+".csv"
p5_file_name= "Slices/p5_"+sector_name+".csv"

with open(p3_file_name, 'w') as f:
    for item in p3.items:
        p3[item].to_csv(f, header=False)

        
with open(p4_file_name, 'w') as g:
    for item in p5.items:
        p4[item].to_csv(g, header=False)


with open(p5_file_name, 'w') as h:
    for item in p5.items:
        p5[item].to_csv(h, header=False)
        
d3=pd.read_csv(p3_file_name, names=main_col_names )
d4=pd.read_csv(p4_file_name, names=main_col_names )
d5=pd.read_csv(p5_file_name, names=main_col_names )

## Making data sane
### 1. Normalizing all data dataset 
### 2. Hot encoding columns

In [234]:
# 1. Normalizing all data dataset 
hot_cols= [
 'monthday',
 'month',
 'weekday',
 'Counter']

# cols_to_normalize=set(p3.minor_axis) - set(hot_cols) - set(["Ticker"]) 
cols_to_normalize=set(d5.columns) - set(hot_cols) - set(["Ticker", "Date"]) 


def df_normalize(df,base_df,cols):
    for col in cols:
        max_v=base_df[col].max()
        min_v=base_df[col].min()
        delta=max_v-min_v
        
        if abs(max_v) and abs (min_v): 
            df[col]=df[col].apply(       lambda x: float(     1*( (x-min_v)/delta )           )   )
        else:
            pass
    return df


def df_normalize_2(df,base_df,cols):
    for col in cols:
        mean_v=base_df[col].mean()
        std_dv=base_df[col].std()
        
        df[col]=df[col].apply(  lambda x: float((   1*(x-mean_v)/std_dv )           )   )
        return df


In [235]:
print cols_to_normalize

set(['EQCAP_Q', 'TotalTradeQuantity', 'ETR_Q', 'MCAP_Q', 'Return', 'OPMSH_Q', 'Turnover_Lacs', 'OPSH1Q_Q', 'DIVSH_Q', 'NP_Q', 'EPS_Q', 'OI_Q', 'INT_Q', 'SR_Q', 'OEXPNS_Q', 'Close', 'PBT_Q', 'TAX_Q', 'PBDT_Q', 'FV_Q', 'OP_Q', 'DEP_Q', 'EBIDT_Q', 'SHARE_Q', 'REVSH_Q', 'TI_Q'])


In [236]:
d3n=d3.copy()
d4n=d4.copy()
d5n=d5.copy()

In [237]:
df_normalize(d3n,d5,cols_to_normalize)
df_normalize(d4n,d5,cols_to_normalize)
df_normalize(d5n,d5,cols_to_normalize)

d5.head()

,Date,Close,TotalTradeQuantity,Turnover_Lacs,Return,DEP_Q,DIVSH_Q,EBIDT_Q,EPS_Q,EQCAP_Q,...,REVSH_Q,SHARE_Q,SR_Q,TAX_Q,TI_Q,monthday,month,weekday,Counter,Ticker
0,2012-09-30,60.9,283315.5,147.1,0.0,0.4,0.3,20.2,-0.3,13.0,...,44.0,1.3,57.6,12.4,59.2,30.0,9.0,7.0,1,NSE/ARIES
1,2012-10-01,62.9,392948.0,238.4,0.2,0.4,0.3,20.1,-0.3,13.0,...,44.0,1.3,57.6,12.4,59.1,1.0,10.0,1.0,2,NSE/ARIES
2,2012-10-02,65.8,502580.8,329.8,0.0,0.4,0.3,20.0,-0.3,13.0,...,43.9,1.3,57.5,12.3,59.1,2.0,10.0,2.0,3,NSE/ARIES
3,2012-10-03,69.7,612214.0,421.3,0.1,0.4,0.3,19.9,-0.3,13.0,...,43.9,1.3,57.4,12.2,59.0,3.0,10.0,3.0,4,NSE/ARIES
4,2012-10-04,69.8,200297.0,143.6,0.0,0.4,0.3,19.8,-0.3,13.0,...,43.8,1.3,57.4,12.1,58.9,4.0,10.0,4.0,5,NSE/ARIES


In [238]:
dummy=pd.get_dummies(d3n, columns = hot_cols)
d3n[dummy.columns]=dummy

dummy=pd.get_dummies(d4n, columns = hot_cols)
d4n[dummy.columns]=dummy

dummy=pd.get_dummies(d5n, columns = hot_cols)
d5n[dummy.columns]=dummy

In [239]:
p5[p5.items[0]]

,Close,TotalTradeQuantity,Turnover_Lacs,Return,DEP_Q,DIVSH_Q,EBIDT_Q,EPS_Q,EQCAP_Q,ETR_Q,...,REVSH_Q,SHARE_Q,SR_Q,TAX_Q,TI_Q,monthday,month,weekday,Counter,Ticker
2012-09-30,60.9,283315.5,147.1,0.0,0.4,0.3,20.2,-0.3,13.0,0.4,...,44.0,1.3,57.6,12.4,59.2,30.0,9.0,7.0,1,NSE/ARIES
2012-10-01,62.9,392948.0,238.4,0.2,0.4,0.3,20.1,-0.3,13.0,0.4,...,44.0,1.3,57.6,12.4,59.1,1.0,10.0,1.0,2,NSE/ARIES
2012-10-02,65.8,502580.8,329.8,0.0,0.4,0.3,20.0,-0.3,13.0,0.4,...,43.9,1.3,57.5,12.3,59.1,2.0,10.0,2.0,3,NSE/ARIES
2012-10-03,69.7,612214.0,421.3,0.1,0.4,0.3,19.9,-0.3,13.0,0.4,...,43.9,1.3,57.4,12.2,59.0,3.0,10.0,3.0,4,NSE/ARIES
2012-10-04,69.8,200297.0,143.6,0.0,0.4,0.3,19.8,-0.3,13.0,0.4,...,43.8,1.3,57.4,12.1,58.9,4.0,10.0,4.0,5,NSE/ARIES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-04-14,197.2,15900.0,32.2,0.0,0.5,0.6,16.5,7.3,13.0,0.3,...,65.8,1.3,84.8,6.2,84.6,14.0,4.0,6.0,104,NSE/ARIES
2018-04-15,195.0,13612.9,27.6,0.0,0.5,0.6,16.5,7.3,13.0,0.3,...,65.8,1.3,84.8,6.2,84.6,15.0,4.0,7.0,105,NSE/ARIES
2018-04-16,192.8,11325.9,23.0,0.0,0.5,0.6,16.5,7.3,13.0,0.3,...,65.8,1.3,84.8,6.2,84.6,16.0,4.0,1.0,106,NSE/ARIES
2018-04-17,190.5,9038.9,18.4,0.0,0.5,0.6,16.5,7.3,13.0,0.3,...,65.8,1.3,84.8,6.3,84.6,17.0,4.0,2.0,107,NSE/ARIES


####  Column identification 


In [240]:
counter_cols=[]
weekday_cols=[]
monthday_cols=[]
month_cols=[]

redunt_cols=["Counter","weekday", "monthday", "month", "Close", "Date", "Ticker"]

# for col in d4.columns:
d5n_cols_filtered=set(d5n.columns) -set(redunt_cols)
d5n_cols_filtered=list(d5n_cols_filtered)

for col in d5n_cols_filtered: 

    if "Return" in col:
        target=col
    
    elif "weekday" in col:
        weekday_cols.append(col)
    elif "monthday" in col:
        monthday_cols.append(col)
    elif "month" in col and "month_day" not in col:
        month_cols.append(col)
    
    elif "Counter" in col:
        counter_cols.append(col)
        
one_hot_cols= counter_cols + weekday_cols + monthday_cols + month_cols  

one_hot_issue= set(one_hot_cols)-set(list(d3n.columns ))

# one_hot_cols= set(one_hot_cols)- one_hot_issue 
one_hot_cols=list(one_hot_cols)

# normalize_cols= set(d5n_cols_filtered) - set(one_hot_cols)
# normalize_cols=list(normalize_cols)

feature_cols= set(d5n_cols_filtered) -set(one_hot_cols) - set(["Return"])
feature_cols=list(feature_cols) 


print("Features:: {}".format(feature_cols))
print("\n\n")
print("Target::   {}".format(target))


Features:: ['EQCAP_Q', 'TotalTradeQuantity', 'ETR_Q', 'MCAP_Q', 'OPMSH_Q', 'PBDT_Q', 'OPSH1Q_Q', 'EPS_Q', 'NP_Q', 'DIVSH_Q', 'OI_Q', 'INT_Q', 'REVSH_Q', 'TI_Q', 'SR_Q', 'PBT_Q', 'TAX_Q', 'Turnover_Lacs', 'FV_Q', 'OP_Q', 'DEP_Q', 'EBIDT_Q', 'SHARE_Q', 'OEXPNS_Q']



Target::   Return


#  ** Starting NN Model from this part **

### Preprocessing Data

In [241]:
def preprocess_features(df):
    
    """
    Parameter:
        Dataframe 
    Return:
        Dataframe: with features only
    """
    
    selected_features = df[feature_cols]
    
#     selected_features = df[one_hot_cols]
    processed_features = selected_features.copy()
    
    
    return processed_features

def preprocess_targets(df):
    """
    Parameter:
        Dataframe:
    Return:
        Datafreme with target values only
    """
    
    output_targets=pd.DataFrame()
    
    output_targets["Return"] = df[target]
    
    
    return output_targets

# Output Feature Column

def construct_feature_columns(input_features):
    return set([
        tf.feature_column.numeric_column(my_feature)
        for my_feature in input_features 
        ])

### Creating Input Function

In [242]:
def my_input_fn(
    features, 
    targets, 
    batch_size=1, 
    shuffle=True, 
    num_epochs=None):
    
        
    
        """ Parameters: 
        features: pd dataframe of feature
        targets: pd dataframe of targets
        batch_size: size of batch
        shuffle shuffle data or not 
        num_epochs:  number of runs 
        
    
        Returns:
        Tuple of (features, lables) for next data batch
        """
        
        # Convert df into dict of np.array.
        features = { 
                    key:np.array(value) 
                    for key, value in dict(features).items() 
                   }
        
        #Contruct a dataset, and configure batching/repaeating.
        ds= Dataset.from_tensor_slices((features, targets))
        ds= ds.batch(batch_size).repeat(num_epochs)
        
        # Shuffle if ordered
        if shuffle:
            ds =ds.shuffle(1000)
            
        
        # Return next batch of the data
        features, labels =ds.make_one_shot_iterator().get_next()
        
        return features, labels

### Creating Training Function

In [243]:
def train_nn_model(
    learning_rate, 
    steps, 
    batch_size,
    hidden_units,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets):

    
    
    # How often you wnat updates:    
    periods =10
    steps_per_period = steps/periods
    
    """ Parameters:
    learning_rate: how big should our step should be 
    steps: how many times should we process batch
    batch_size: sample size 
    input_feature: what should be the input 
    """
    
    #Configure regressor:
    my_optimizer= tf.train.AdamOptimizer(learning_rate= learning_rate)
    dnn_regressor = tf.estimator.DNNRegressor(
        feature_columns=construct_feature_columns(training_examples),
        hidden_units=hidden_units,
        optimizer=my_optimizer,
        activation_fn=tf.nn.relu
            
        )
    
    # Creating input pfunction 
    
    training_input_fn = lambda: my_input_fn(
        training_examples,
        training_targets["Return"],
        batch_size=batch_size)
    
    predict_training_input_fn = lambda: my_input_fn(
        training_examples,
        training_targets["Return"],
        num_epochs=1,
        shuffle=False)
    
    predict_validation_input_fn = lambda: my_input_fn(
        validation_examples,
        validation_targets["Return"],
        num_epochs=1,
        shuffle=False)
        
    
    

    # Trainign model from below code
    print "Training started:"
    print  "RMSE on models are:"
    training_rmses=[]
    validation_rmses=[]
   
    for  period in  range(0, periods):
        dnn_regressor.train(
            input_fn = training_input_fn,
            steps = steps_per_period
            
        )
        
    
        # Take a break and and compute some predictions 
    
        training_predictions =dnn_regressor.predict(input_fn=predict_training_input_fn)
        training_predictions = np.array([
                            item["predictions"][0]
                            for item in training_predictions 
                           ])
    
        validation_predictions =dnn_regressor.predict(input_fn=predict_validation_input_fn)
        validation_predictions = np.array([
                            item["predictions"][0]
                            for item in validation_predictions 
                           ])
    
        
        
        # Compute loss
        training_mse=metrics.mean_squared_error(training_predictions, training_targets)
        training_rmse=math.sqrt( training_mse )
        
        validation_mse=metrics.mean_squared_error(validation_predictions, validation_targets)
        validation_rmse=math.sqrt( validation_mse )
        
        
        #Print current loss
        print "period %02d: %0.2f" % (period, training_rmse)
        
        #Save rmse
        training_rmses.append(training_rmse)
        validation_rmses.append(validation_rmse)
       
    
    print "Model training finished."
    
    # Output graph of loss metrics over periods. 
    plt.ylabel('RMSE')
    plt.xlabel("Periods")
    plt.title("RMSE vs Periods")
    plt.tight_layout()
    plt.plot(training_rmses, label="training")
    plt.plot(validation_rmses, label="validation")
    plt.legend()
  

    return  dnn_regressor

### Spliting Dataset

In [244]:
## Floating error avoadance:

length_of_df=int(len(d3n))
r=.9


# Using rargets  tio factor r
training_count   = int(r*length_of_df)  
validation_count = length_of_df-training_count

#Shuffle all the valaues 
d3n=d3n.reindex(np.random.permutation(d3n.index))

training_examples   = preprocess_features(d3n.head(training_count))
training_targets    = preprocess_targets( d3n.head(training_count))

validation_examples = preprocess_features(d3n.tail(validation_count))
validation_targets  = preprocess_targets( d3n.tail(validation_count))

### Training Model

In [ ]:
dnn_regressor= train_nn_model(learning_rate=0.0000001, steps=1000000, batch_size=5000, hidden_units=[24,20,20],
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets
    )

Training started:
RMSE on models are:
period 00: 0.25


## Predcicting stock returns

In [ ]:
stock_examples = preprocess_features(d4n)
stock_targets  = preprocess_targets(d4n)

predict_stock_input_fn = lambda: my_input_fn(
        stock_examples,
        stock_targets["Return"],
        num_epochs=1,
        shuffle=False)

stock_predictions =dnn_regressor.predict(input_fn=predict_stock_input_fn)
stock_predictions = np.array([
                        item["predictions"][0]
                        for item in stock_predictions 
                       ])

print list(stock_predictions)

## Saving prediction to dataframe

In [ ]:
d6_cols=d4["Ticker"].unique()
d6_index=d4["Date"].unique()

d6=pd.DataFrame(columns=d6_cols, index=d6_index)

j=0
for tick in d6.columns: 
    d6[tick]=stock_predictions[j:j+7]
    j+=7
d6

## Denormalizing Data

In [ ]:
def denoramlize(df, base_df, col):
    
    min_v=min(base_df[col]) 
    max_v=max(base_df[col])
    delta = max_v-min_v

    print("Min:  {}".format(min_v))
    print("Max:  {}".format(max_v))
    print("delta:{}".format(delta))


    for col in df.columns:
        df[col]=df[[col]].apply(lambda x: min_v + (x*(delta/1)) )
        
def denoramlize_2(df, base_df, col):
    
    mean_v=base_df[col].mean() 
    std_dv=max(base_df[col])
    
    print("Mean  :  {}".format(mean_v))
    print("Std_dv:  {}".format(std_dv))
    


    for col in df.columns:
        df[col]=df[[col]].apply(lambda x: mean_v + ((x*std_dv)/1) )        
        
        
        

In [ ]:
denoramlize(d6, d5, "Return")


### Saving Data..... 


In [ ]:
sector_name=w.kwargs["sector_name"]

result_file_name= "../Panel/Result/Predicted/"+sector_name+".csv"

d6.to_csv(result_file_name) 